In [25]:
import requests

url = "https://www.psl.noaa.gov/enso/mei/data/meiv2.data"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the data to a file
    with open('meiv2.data', 'w') as file:
        file.write(response.text)
else:
    print("Failed to retrieve data")


In [38]:
import pandas as pd

# Read the data into a DataFrame, delim tab, no header, last lines delete, skip first line
enso_mei = pd.read_csv('meiv2.data', delim_whitespace=True, header=None, skipfooter=3, engine='python', skiprows=1)
#Set first column as index as year, remove 0 column index
enso_mei = enso_mei.set_index(0)
enso_mei.index.name = None
#Set column names as bi monthly periods (Dec/Jan, Jan/Feb, etc)
enso_mei.columns = ['Dec-Jan', 'Jan-Feb', 'Feb-Mar', 'Mar-Apr', 'Apr-May', 'May-Jun', 'Jun-Jul', 'Jul-Aug', 'Aug-Sep', 'Sep-Oct', 'Oct-Nov', 'Nov-Dec']
#Remove last row and all -999.00 values
enso_mei = enso_mei.iloc[:-1]
enso_mei = enso_mei.replace(-999.00, None)

display(enso_mei)


,Dec-Jan,Jan-Feb,Feb-Mar,Mar-Apr,Apr-May,May-Jun,Jun-Jul,Jul-Aug,Aug-Sep,Sep-Oct,Oct-Nov,Nov-Dec
1979.0,0.47,0.27,-0.04,0.26,0.35,-0.14,-0.13,0.50,0.42,0.28,0.58,0.68
1980.0,0.42,0.30,0.52,0.85,0.71,0.55,0.52,0.13,0.19,0.06,-0.09,-0.09
1981.0,-0.28,-0.19,0.16,0.15,-0.38,-0.63,-0.51,-0.25,0.03,-0.05,-0.16,-0.18
1982.0,-0.44,-0.61,-0.31,0.13,0.66,1.42,1.87,1.93,1.74,1.87,2.25,2.45
1983.0,2.59,2.73,2.71,2.74,2.60,1.58,0.88,0.03,-0.41,-0.44,-0.42,-0.44
1984.0,-0.38,-0.36,-0.06,-0.06,-0.78,-0.75,-0.23,-0.02,-0.06,-0.01,-0.59,-0.32
1985.0,-0.31,-0.57,-0.51,-0.68,-1.04,-0.24,-0.02,-0.36,-0.35,0.05,0.11,-0.36
1986.0,-0.42,-0.33,-0.37,-0.30,0.24,0.61,0.77,1.24,1.38,0.57,0.73,1.07
1987.0,0.96,1.12,1.49,1.62,1.85,1.78,1.74,1.49,1.18,1.05,0.89,0.86
1988.0,0.64,0.39,0.27,-0.13,-0.59,-1.35,-1.77,-1.82,-1.8,-1.53,-1.65,-1.56
